1. 设计一个具有单隐层的多层感知器网络；
2. 构造多层感知器前向传播和后向传播程序；
3. 训练多层感知器；
4. 利用多层感知器实现 MINIST 手写体识别；

# 1 数据准备

## 1-1 超参数

In [ ]:
batch_size = 64  # 每次训练时使用的样本数量。将数据分成小批次，可以减少内存占用，同时加速训练。
learning_rate = 0.01
num_epochs = 10

## 1-2 数据预处理

In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.ToTensor(),  # 将图像数据从 PIL 图像或 numpy 数组转换为 PyTorch 的张量（tensor）。MNIST 数据集中的每个图像是 28x28 像素的灰度图，ToTensor 会将其转换为 [1, 28, 28] 的张量，并且像素值会被缩放到 [0, 1] 之间。
    transforms.Normalize((0.5,), (0.5,))  # 归一化（Normalization）操作。这里 (0.5,) 表示均值，(0.5,) 表示标准差。因为 MNIST 是灰度图，只有一个通道，所以只有一个值。数据归一化的作用是将输入数据的值调整到一个标准的范围内，有助于加快收敛速度和提升模型性能。具体来说，这里的操作是将数据缩放到 [-1, 1] 之间
])

## 1-3 加载训练集和测试集

In [ ]:
from torchvision import datasets

train_dataset = datasets.MNIST(root='../data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='../data', train=False, download=True, transform=transform)

## 1-4 创建数据加载器

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# 2 定义模型

In [ ]:
import torch
from torch import nn

class MyMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.cf1 = nn.Linear(28*28, 128)
        self.cf2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.cf1(x))
        x = self.cf2(x)
        return x

# 初始化模型
model = MyMLP()

# 3 定义损失函数和优化器

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 4 训练模型

In [ ]:
model.train()
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 后向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f'Epoch: {epoch+1}; Step: {i+1}; Loss: {loss.item():.4f}.')  # 打印损失

# 5 评估模型

In [ ]:
model.eval()  # 切换到评估模式
correct = 0
total = 0

with torch.no_grad():  # 不需要计算梯度
    for images, labels in test_loader:
        outputs = model(images)  # 前向传播
        _, predicted = torch.max(outputs.data, 1)  # 返回两个值：一个是每个样本的最大值（在这里我们不需要），另一个是最大值的索引（也就是预测的类别）。
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = correct / total

print(f'Accuracy: {accuracy*100:.2f}%.')  # 打印准确率

# 6 预测单个样本（附加）

In [ ]:
import matplotlib.pyplot as plt

# 从测试集中取一个样本
sample = next(iter(test_loader))
images, labels = sample

# 取第一张图片进行预测
image = images[0].unsqueeze(0)  # 增加一个维度，适应模型输入
model.eval()
output = model(image)
_, predicted = torch.max(output.data, 1)

# 可视化样本及其预测结果
plt.imshow(image.squeeze(), cmap='gray')
plt.title(f'Predicted: {predicted.item()}')
plt.show()